### Load Modules


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import h5py 
import sklearn

### Load all the ground truth and prediction data  

In [ ]:
cls_gt = pd.read_csv('../experiments/outputs/cls_gt.csv', header=None)
cls_pred = pd.read_csv('../experiments/outputs/cls_pred.csv', header=None)
doa_gt = pd.read_csv('../experiments/outputs/doa_gt.csv', header=None)
doa_pred = pd.read_csv('../experiments/outputs/doa_pred.csv', header=None)

In [ ]:
cls_pred_cleaned = cls_pred.applymap(lambda x: 1.0 if x > 0.3 else 0.0)
cls_pred_cleaned

In [ ]:
identical_rows = (cls_gt == cls_pred_cleaned).all(axis=1)
num_identical_rows = identical_rows.sum()
print(num_identical_rows)

In [ ]:
identical_rows[0]

In [ ]:
for i in range(3):
    doa_gt[f'az{i}'] = np.degrees(np.arctan2(doa_gt[i], doa_gt[i+3]))
    doa_gt[f'az{i}'] = (doa_gt[f'az{i}'] + 360)%360
    
    doa_pred[f'az{i}'] = np.degrees(np.arctan2(doa_pred[i], doa_pred[i+3]))
    doa_pred[f'az{i}'] = (doa_pred[f'az{i}'] + 360)%360

In [ ]:
azi_gt = doa_gt[['az0' , 'az1' , 'az2']].copy()
azi_pred = doa_pred[['az0' , 'az1' , 'az2']].copy()

In [ ]:
gt_azi = azi_gt.values.tolist()
pred_azi = azi_pred.values.tolist()

In [ ]:

mask = [[1 if value !=0 else 0 for value in sublist] for sublist in gt_azi]
pred_arr = np.array(pred_azi)
mask_arr = np.array(mask)
azi_gt_arr = np.array(gt_azi)
masked_pred = list(pred_arr * mask_arr)

In [ ]:
mean_abs_diff = np.mean(np.abs(masked_pred - azi_gt_arr))*3
print(mean_abs_diff)

In [ ]:
np.abs(masked_pred[2] - azi_gt_arr[2])

In [ ]:
np.sum(np.abs(masked_pred[2] - azi_gt_arr[2])) 

### Location dependent error rate

In [ ]:
is_class_correct = identical_rows.values.tolist()
count = 0
check_cls = 0
neg_count = 0
for i in range(len(is_class_correct)):
    if is_class_correct[i]:
        check_cls += 1
        deg_diff = np.sum(np.abs(masked_pred[i] - azi_gt_arr[i]))
        if deg_diff <= 20:
            count += 1
        else:
            neg_count += 1
          
print(check_cls)  
print(count)
print(neg_count)

In [ ]:
azi_gt

In [ ]:
masked_pred

In [ ]:
for i in range(len(masked_pred)):
    print(azi_gt_arr[i])
    print(masked_pred[i])
    print('\n')